<a href="https://colab.research.google.com/github/saklain31/sentence_classification/blob/master/Bangla_sentence_classification_using_word2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import nltk
import gensim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report,accuracy_score
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.metrics import accuracy_score, precision_score, f1_score, log_loss
from sklearn.metrics import classification_report,confusion_matrix, precision_recall_fscore_support 

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
model = gensim.models.Word2Vec.load('drive/My Drive/NLPDataset/banglaData')
vocab = list(model.wv.vocab.keys())

In [0]:
wv = model
wv.init_sims(replace=True)
from itertools import islice
list(islice(vocab, 13030, 13050))

In [0]:
import pandas as pd
data = pd.read_csv('drive/My Drive/NLPDataset/banglaTemp.csv', sep='\t',engine='python')
datalist = data.values.tolist()

# print(type(datalist))

questions = []
intents = []

ln = len(datalist)
for i in range(ln):
  questions.append(datalist[i][0])
  intents.append(datalist[i][1])


In [0]:
def get_sentence_to_vecAvg(model,sentence):
  sentence = sentence.split()
  #print(sentence)
  
  vecArray = []
  for i in sentence:
    if i in model.wv.vocab:
      vecArray.append(model.wv[i])
  
  avg = gensim.matutils.unitvec(np.array(vecArray).mean(axis=0)).astype(np.float32)
  return avg


In [0]:
vec_questions = []
for i in questions:
  vec_questions.append(get_sentence_to_vecAvg(model,i))
  
  
X_train, X_test, y_train, y_test = train_test_split(vec_questions, intents, test_size=0.4, random_state = 42)


ln = len(X_test)

X_test_val = X_test[int(ln/2):ln]
X_test = X_test[0:int(ln/2)]

y_test_val = y_test[int(ln/2):ln]
y_test = y_test[0:int(ln/2)]


In [0]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train, y_train)


y_pred = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

y_pred_val = logreg.predict(X_test_val)
print('accuracy %s' % accuracy_score(y_pred_val, y_test_val))
print(classification_report(y_test_val, y_pred_val))

In [0]:
clf = NuSVC(nu=.09, degree=5, kernel='rbf', gamma='scale',probability=True)


In [0]:
clf.fit(X_train, y_train)

#TEST SET
try:
  print('****Results****')
#   print(X_test)
  y_pred = clf.predict(X_test)
  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred))

except Exception as e:
  print (e)
  
#VALIDATION SET  
try:
  print('****Results****')
  y_pred_val = clf.predict(X_test_val)
  print('accuracy %s' % accuracy_score(y_pred_val, y_test_val))
  print(classification_report(y_test_val, y_pred_val))
  
except Exception as e:
  print (e)

In [0]:
text = input()
print(logreg.predict([get_sentence_to_vecAvg(model,text)])[0])
print(clf.predict([get_sentence_to_vecAvg(model,text)])[0])